In [3]:
import pandas as pd
import datetime as dt
from pandas_datareader import data


In [4]:
df = pd.read_csv("files/input (1).csv",parse_dates=['date'], dayfirst=True)
# df['date'] = df['date'].apply(lambda x: x.date())
df.sort_values(by='date').dropna()

,date,stock,action,qty,price
0,2020-10-26,Cash,Deposit,1,100000
1,2020-10-27,Cash,Deposit,1,1000
2,2020-10-28,AAPL,BUY,10,100
3,2020-10-29,TSLA,BUY,10,600
4,2020-10-30,SPY,BUY,10,300
5,2020-10-31,Cash,Rebate,1,2
6,2020-11-01,Cash,Rebate,1,2
7,2020-11-02,Cash,Rebate,1,2
8,2020-11-03,AAPL,SELL,-5,120
9,2020-11-04,TSLA,SELL,-2,700


In [247]:
def get_data(df):
    '''
    Will return positions_df, realised_gains, unrealised_gains, portfolio_size, available_cash
    '''  
    df_positions = df.copy(deep=True)
    sells = df_positions[df_positions['action'] == 'SELL'].sort_values(by='date').reset_index()
    exclude_sells = df_positions[df_positions['action'] != 'SELL'].sort_values(by='date').reset_index()
    realised_gains = 0

    # Process first in first out 
    for idx, row in sells.iterrows():

        stock = row['stock']
        count = row['qty']
        sell_price = row['price']
        while abs(count) > 0:
            
            # Find first occurance of stock
            first_index = (exclude_sells.stock.values == stock).argmax()
#             print(exclude_sells)
#             print(stock)
#             print(first_index)
#             print(exclude_sells.at[first_index,'qty'])
#             print(count)

            # Exact amount
            if exclude_sells.iloc[first_index]['qty'] == abs(count):
                exclude_sells.drop([first_index])
                realised_gains += (sell_price - exclude_sells.at[first_index,'price']) * count
                count = 0
            # Enough to sell
            elif exclude_sells.iloc[first_index]['qty'] > abs(count):
                exclude_sells.at[first_index,'qty'] += count
                realised_gains += (sell_price - exclude_sells.at[first_index,'price']) * count
                count = 0
            # Not enough
            else:
                exclude_sells.drop([first_index])
                realised_gains += (sell_price - exclude_sells.at[first_index,'price']) * exclude_sells.at[first_index,'qty']
                count += exclude_sells.at[first_index,'qty']

    # Find the current positions
    positions = {}
    buys = exclude_sells[exclude_sells['action'] == 'BUY'].sort_values(by='date')
    for idx,row in buys.iterrows():

        stock = row['stock']
        qty = int(row['qty'])
        price = int(row['price'])

        if stock not in positions.keys():
            positions[stock] = [qty,price]
        else:
            new_qty = qty + positions[stock][0]
            new_price = ((qty*price) + (positions[stock][0]*positions[stock][1])) / new_qty
            positions[stock] = [new_qty,new_price]
            
    positions_df = pd.DataFrame(data=positions, index=['qty','price']).T.reset_index().rename(columns={'index':'stock'})

    # Add in current prices

    tickers = list(df.stock.unique())
    tickers.remove('Cash')

    date = dt.date.today() - dt.timedelta(days=1)

    panel_data = data.DataReader(tickers, 'yahoo', date, date)

    current_prices = []

    for idx, row in positions_df.iterrows():

        stock = row['stock']

        price = panel_data['Close'][stock].tail(1)[0]
        current_prices.append(price)

    positions_df['current_prices'] = current_prices

    # Adding in floating profits
    positions_df['P&L'] = (positions_df['current_prices'] -
                           positions_df['price']) * positions_df['qty']

    # Realised gains, unrealised, portfolio size, available cash
    print(realised_gains)
    unrealised_gains = positions_df['P&L'].sum()
    print(unrealised_gains)
    portfolio_size = df[df['action'] == 'Deposit']['price'].astype(
        'int').sum() - df[df['action'] == 'Withdraw']['price'].astype('int').sum()
    print(portfolio_size)
    available_cash = df[df['action'] == 'Deposit']['price'].astype(
        'int').sum() - (positions_df['price'] * positions_df['qty']).sum()
    print(available_cash)
    positions_df.round(3)
    
    return positions_df, realised_gains, unrealised_gains, portfolio_size, available_cash

In [248]:
positions_df, realised_gains, unrealised_gains, portfolio_size, available_cash = get_data(df)
# positions_df.info()
positions_df

C:\Users\Razer\anaconda3\lib\site-packages\pandas_datareader\base.py:272: SymbolWarning: Failed to read symbol: 'LCID ', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
C:\Users\Razer\anaconda3\lib\site-packages\pandas_datareader\base.py:272: SymbolWarning: Failed to read symbol: 'Cash ', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
C:\Users\Razer\anaconda3\lib\site-packages\pandas_datareader\base.py:272: SymbolWarning: Failed to read symbol: 'MU ', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)


-499440
-17050.833967924118
101190
45190.0


,stock,qty,price,current_prices,P&L
0,AAPL,16.0,105.000000,143.304596,612.873535
1,TSLA,18.0,711.111111,777.109985,1187.979736
2,SPY,24.0,260.833333,434.904999,4177.719971
3,NFLX,13.0,35.384615,601.859985,7364.179810
4,SLV,8.0,310.000000,19.945000,-2320.440002
5,OPTT,10.0,200.000000,2.460000,-1975.400000
6,USWS,10.0,540.000000,1.100500,-5388.995000
7,PROG,20.0,50.000000,1.065000,-978.699999
8,NAVI,2.0,100.000000,19.865000,-160.270000
9,HLBZ,10.0,400.000000,13.530100,-3864.699001


In [187]:
df_positions = df.copy(deep=True)
sells = df_positions[df_positions['action'] == 'SELL'].sort_values(by='date')
exclude_sells = df_positions[df_positions['action'] != 'SELL'].sort_values(by='date')
realised_gains = 0

# Process first in first out 
for idx, row in sells.iterrows():
    
    stock = row['stock']
    count = row['qty']
    sell_price = row['price']
    while abs(count) > 0:
        
        # Find first occurance of stock
        first_index = (exclude_sells.stock.values == stock).argmax()
        print(stock, first_index)        
        
        # Exact amount
        if exclude_sells.iloc[first_index]['qty'] == abs(count):
            exclude_sells.drop([first_index])
            realised_gains += (sell_price - exclude_sells.at[first_index,'price']) * count
            count = 0
        # Enough to sell
        elif exclude_sells.iloc[first_index]['qty'] > abs(count):
            exclude_sells.at[first_index,'qty'] += count
            realised_gains += (sell_price - exclude_sells.at[first_index,'price']) * count
            count = 0
        # Not enough
        else:
            exclude_sells.drop([first_index])
            realised_gains += (sell_price - exclude_sells.at[first_index,'price']) * exclude_sells.at[first_index,'qty']
            count += exclude_sells.at[first_index,'qty']
            
# Find the current positions
positions = {}
buys = exclude_sells[exclude_sells['action'] == 'BUY'].sort_values(by='date')
for idx,row in buys.iterrows():
    
    stock = row['stock']
    qty = int(row['qty'])
    price = int(row['price'])
    
    if stock not in positions.keys():
        positions[stock] = [qty,price]
    else:
        new_qty = qty + positions[stock][0]
        new_price = ((qty*price) + (positions[stock][0]*positions[stock][1])) / new_qty
        positions[stock] = [new_qty,new_price]
        
# Check
exclude_sells.sort_values(by='date').head(10)
positions_df = pd.DataFrame(data=positions, index=['qty','price']).T.reset_index().rename(columns={'index':'stock'})
print(realised_gains)
positions_df

AAPL 2
TSLA 3
-300.0


,stock,qty,price
0,AAPL,15.0,126.666667
1,TSLA,18.0,711.111111
2,SPY,10.0,300.000000


In [200]:
# Add in current prices

tickers = list(df.stock.unique())
tickers.remove('Cash')

date = dt.date.today() - dt.timedelta(days=1)

panel_data = data.DataReader(tickers, 'yahoo', date, date)

current_prices = []

for idx, row in positions_df.iterrows():

    stock = row['stock']

    price = panel_data['Close'][stock].tail(1)[0]
    current_prices.append(price)

positions_df['current_prices'] = current_prices

# Adding in floating profits
positions_df['P&L'] = (positions_df['current_prices'] -
                       positions_df['price']) * positions_df['qty']

# Realised gains, unrealised, portfolio size, available cash
print(realised_gains)
unrealised_gains = positions_df['P&L'].sum()
print(unrealised_gains)
portfolio_size = df[df['action'] == 'Deposit']['price'].astype(
    'int').sum() - df[df['action'] == 'Withdraw']['price'].astype('int').sum()
print(portfolio_size)
avilable_cash = df[df['action'] == 'Deposit']['price'].astype(
    'int').sum() - (positions_df['price'] * positions_df['qty']).sum()
print(avilable_cash)
positions_df.round(3)

-300.0
2682.497222900391
1000
-15700.0


,stock,qty,price,current_prices,P&L
0,AAPL,15.0,126.667,146.47,297.050
1,TSLA,18.0,711.111,764.03,952.547
2,SPY,10.0,300.000,443.29,1432.900


In [2]:
import sys
sys.path.insert(1, '/notebooks')
from ipynb.fs.full.volatility_sharpe_ratio import *


ModuleNotFoundError: No module named 'ipynb'